In [43]:
import requests
import json
import time
import pandas as pd

# Token accessToken 及权限校验机制
getAccessTokenUrl = 'https://quantapi.51ifind.com/api/v1/get_access_token'
# 获取refresh_token需下载Windows版本接口包解压，打开超级命令-工具-refresh_token查询
with open ('token.txt', 'r') as file:
    refreshtoken = file.read().strip()
getAccessTokenHeader = {"Content-Type": "application/json", "refresh_token": refreshtoken}
getAccessTokenResponse = requests.post(url=getAccessTokenUrl, headers=getAccessTokenHeader)
accessToken = json.loads(getAccessTokenResponse.content)['data']['access_token']
print(accessToken)

thsHeaders = {"Content-Type": "application/json", "access_token": accessToken}


# 高频序列：获取分钟数据
def high_frequency():
    thsUrl = 'https://quantapi.51ifind.com/api/v1/high_frequency'
    thsPara = {"codes":
                   "000001.SZ",
               "indicators":
                   "open,high,low,close,volume,amount,changeRatio",
               "starttime":
                   "2022-07-05 09:15:00",
               "endtime":
                   "2022-07-05 15:15:00"}
    thsResponse = requests.post(url=thsUrl, json=thsPara, headers=thsHeaders)
    print(thsResponse.content)


# 实时行情：循环获取最新行情数据
def real_time():
    thsUrl = 'https://quantapi.51ifind.com/api/v1/real_time_quotation'
    thsPara = {"codes": "300033.SZ", "indicators": "latest"}
    while True:
        thsResponse = requests.post(url=thsUrl, json=thsPara, headers=thsHeaders)
        data = json.loads(thsResponse.content)
        result = pd.json_normalize(data['tables'])
        result = result.drop(columns=['pricetype'])
        result = result.apply(lambda x: x.explode().astype(str).groupby(level=0).agg(", ".join))
        print(result)
        # do your thing here
        time.sleep(3)
        pass


# 历史行情：获取历史的日频行情数据
def history_quotes():
    thsUrl = 'https://quantapi.51ifind.com/api/v1/cmd_history_quotation'
    thsPara = {"codes":
                   "000001.SZ,600000.SH",
               "indicators":
                   "open,high,low,close",
               "startdate":
                   "2021-07-05",
               "enddate":
                   "2022-08-05",
               "functionpara":
                   {"Fill": "Blank"}
               }
    thsResponse = requests.post(url=thsUrl, json=thsPara, headers=thsHeaders)
    data = json.loads(thsResponse.content)
    table = data.get('tables', [{}])[0]
    columns = table.get('columns', [])
    rows = table.get('rows', [])
    df = pd.DataFrame(rows, columns=columns)
    return df


# 基础数据：获取证券基本信息、财务指标、盈利预测、日频行情等数据
def basic_data():
    thsUrl = 'https://quantapi.51ifind.com/api/v1/basic_data_service'
    thsPara = {"codes":
                   "300033.SZ,600000.SH",
               "indipara":
                   [
                       {
                           "indicator":
                               "ths_regular_report_actual_dd_stock",
                           "indiparams":
                               ["104"]
                       },
                       {
                           "indicator":
                               "ths_total_shares_stock",
                           "indiparams":
                               ["20220705"]
                       }
                   ]
               }
    thsResponse = requests.post(url=thsUrl, json=thsPara, headers=thsHeaders)
    print(thsResponse.content)


# 日期序列：与基础数据指标相同，可以同时获取多日数据
def date_serial():
    thsUrl = 'https://quantapi.51ifind.com/api/v1/date_sequence'
    thsPara = {"codes":
                   "000001.SZ,600000.SH",
               "startdate":
                   "20220605",
               "enddate":
                   "20220705",
               "functionpara":
                   {"Fill": "Blank"},
               "indipara":
                   [
                       {
                           "indicator":
                               "ths_close_price_stock",
                           "indiparams":
                               ["", "100", ""]
                       },
                       {"indicator":
                            "ths_total_shares_stock",
                        "indiparams":
                            [""]
                        }
                   ]
               }
    thsResponse = requests.post(url=thsUrl, json=thsPara, headers=thsHeaders)
    data = json.loads(thsResponse.content)
    print(data)


# 专题报表，示例提取全部A股代码，更多报表数据使用超级命令工具查看
def data_pool():
    thsUrl = 'https://quantapi.51ifind.com/api/v1/data_pool'
    thsPara = {
        "reportname": "p03425",
        "functionpara": {
            "date": "20220706",
            "blockname": "001005010",
            "iv_type": "allcontract"
        },
        "outputpara": "p03291_f001,p03291_f002,p03291_f003,p03291_f004"
    }
    thsResponse = requests.post(url=thsUrl, json=thsPara, headers=thsHeaders)
    print(thsResponse.content)


# 经济数据库
def edb():
    thsUrl = 'https://quantapi.51ifind.com/api/v1/edb_service'
    thsPara = {"indicators":
                   "G009035746",
               "startdate":
                   "2022-04-01",
               "enddate":
                   "2022-05-01"}
    thsResponse = requests.post(url=thsUrl, json=thsPara, headers=thsHeaders)
    print(thsResponse.content)


# 日内快照：tick数据
def snap_shot():
    thsUrl = 'https://quantapi.51ifind.com/api/v1/snap_shot'
    thsPara = {
        "codes": "000001.SZ",
        "indicators": "open,high,low,latest,bid1,ask1,bidSize1,askSize1",
        "starttime": "2022-07-06 09:15:00",
        "endtime": "2022-07-06 15:15:00"
    }
    thsResponse = requests.post(url=thsUrl, json=thsPara, headers=thsHeaders)
    print(thsResponse.content)


# 公告函数
def report_query():
    thsUrl = 'https://quantapi.51ifind.com/api/v1/report_query'
    thsPara = {
        "codes": "000001.SZ,600000.SH",
        "functionpara": {
            "reportType": "901"
        },
        "beginrDate": "2021-01-01",
        "endrDate": "2022-07-06",
        "outputpara": "reportDate:Y,thscode:Y,secName:Y,ctime:Y,reportTitle:Y,pdfURL:Y,seq:Y"
    }
    thsResponse = requests.post(url=thsUrl, json=thsPara, headers=thsHeaders)
    print(thsResponse.content)


# 智能选股
def WCQuery():
    thsUrl = 'https://quantapi.51ifind.com/api/v1/smart_stock_picking'
    thsPara = {
        "searchstring": "涨跌幅",
        "searchtype": "stock"
    }
    thsResponse = requests.post(url=thsUrl, json=thsPara, headers=thsHeaders)
    print(thsResponse.content)


# 日期查询函数、日期偏移函数：根据交易所查询交易日
def date_offset():
    thsUrl = 'https://quantapi.51ifind.com/api/v1/get_trade_dates'
    thsPara = {"marketcode": "212001",
               "functionpara":
                   {"dateType": "0",
                    "period": "D",
                    "offset": "-10",
                    "dateFormat": "0",
                    "output": "sequencedate"},
               "startdate":
                   "2022-07-05"}
    thsResponse = requests.post(url=thsUrl, json=thsPara, headers=thsHeaders)
    print(thsResponse.content)


# def main():
    # 实时行情
    # real_time()
    # 基础数据
    # basic_data()
    # 日期序列
    # date_serial()
    # 专题报表
    # data_pool()
    # 历史行情
    history_quotes()
    # 高频序列
    # high_frequency()
    # 经济数据库
    # edb()
    # 日内快照
    # snap_shot()
    # 公告函数
    # report_query()
    # 智能选股
    # WCQuery()
    # 日期查询函数
    # date_query()
    # 日期偏移函数
    # date_offset()


4222a95b2ee94a2b1910d968467947b745562266.signs_NzMwMTMyNTUw


In [46]:
data = history_quotes()

In [49]:
df

,thscode,time,table.open,table.high,table.low,table.close
0,000001.SZ,"[2021-07-05, 2021-07-06, 2021-07-07, 2021-07-0...","[21.69, 22.08, 22.68, 22.78, 21.3, 21.5, 21.15...","[22.15, 23, 23.05, 22.78, 21.63, 21.5, 21.34, ...","[21.09, 21.85, 22.5, 21.2, 21.02, 20.89, 20.54...","[22.06, 22.78, 22.55, 21.51, 21.27, 21.16, 21...."
1,600000.SH,"[2021-07-05, 2021-07-06, 2021-07-07, 2021-07-0...","[9.95, 9.91, 10.01, 10.05, 9.93, 9.96, 9.94, 9...","[9.96, 10.01, 10.04, 10.05, 10.01, 9.99, 9.99,...","[9.85, 9.88, 9.94, 9.92, 9.9, 9.9, 9.88, 9.8, ...","[9.92, 9.99, 10.03, 9.94, 9.91, 9.91, 9.96, 9...."
